In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
def extract_page(pesquisa,page=1):
    p = pesquisa.split()
    p1 = '-'.join(p)
    p2 = '%20'.join(p)
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Safari/537.36'}
    url = f'https://www.catho.com.br/vagas/{p1}/?q={p2}&page={page}'
    session = requests.get(url,headers)
    soup = BeautifulSoup(session.content,'html.parser')
  
    
    return soup

def transform_page(soup):
    data = {'title':[],'descrição':[],'local':[],'salario':[],'link':[]}
    
    articles = soup.find_all('article')
    for art in articles[3::]:
        try:    data['title'].append(art.h2.text)
        except: data['title'].append('')
            
        try:    data['descrição'].append(art.find('span',{'class':"job-description"}).text)
        except: data['descrição'].append('')
            
        try:    data['local'].append(art.find('button',{'class':"sc-ksYbfQ ehLxPn"}).text)
        except: data['local'].append('')
            
        try:    data['salario'].append(str(art.find('div',{'class':"sc-hqyNC exuLzT"}).text.replace('R$','')))
        except: data['salario'].append('')
            
        try:    data['link'].append(art.a.attrs['href'])
        except: data['link'].append('')
            
    return pd.DataFrame(data)

def get_descrição_completa(links):
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Safari/537.36'}
    desc_completa = []
    
    for link in links:
        session = requests.get(link,headers)
        soup = BeautifulSoup(session.content,'html.parser')
        
        try:    
            desc_string = soup.find('span',{'class':'job-description'}).text
            desc_completa.append(desc_string.replace('\n','').replace('  ',''))
                
        except: 
            desc_completa.append('')

    return  desc_completa  

In [8]:
%%time
area = 'Analista fiscal tributário'
soup = extract_page(area)
df_all = transform_page(soup)
    
for i in range(1,40):
    soup = extract_page(area,i)
    df = transform_page(soup)
    df_all = pd.concat([df_all,df],ignore_index=True)



Wall time: 19.7 s


In [27]:
#df_all.to_excel('Catho Analistas controladoria.xlsx')

In [9]:
links = df_all['link']

In [5]:
%%time
all_description = get_descrição_completa(links)

Wall time: 12min 22s


In [6]:
df_all['descrição completa'] = all_description
df_all = df_all.dropna()

In [7]:
df_all.to_excel('Catho Analistas contábil.xlsx')

In [10]:
%%time
all_description = get_descrição_completa(links)

Wall time: 9min 11s


In [11]:
df_all['descrição completa'] = all_description
df_all = df_all.dropna()

In [12]:
df_all.to_excel('Catho Analistas fiscal.xlsx')